### Calculate distance away from the surface normals 

In [1]:
# Import required libraries
import numpy as np
import nibabel as nib
from scipy.ndimage import map_coordinates
import os
import matplotlib.pyplot as plt
import re
from scipy.stats import zscore
from qc_util import *


#### run calculate surface normals and c oordinates using the qc_util.py function before executing the next part
#### and calculate surface with X spacing along the surface normals 

In [2]:
print(nib)

<module 'nibabel' from '/opt/anaconda3/envs/niwrap3912/lib/python3.11/site-packages/nibabel/__init__.py'>


In [ ]:
# 1. Calculate surface normals and coordinates
DATA_TYPE = 'exvivo_final'
DATA_PATH = "/Users/dennis.jungchildmind.org/Desktop/To-be-uploaded-to-CMI-server/data/thickness_data/"

calculate_surfnorm_and_coords(DATA_TYPE, DATA_PATH)

# 2. Calculate surface with X spacing along the surface normals 
path = f"{DATA_PATH}/{DATA_TYPE}"
save_path = f"{DATA_PATH}/{DATA_TYPE}/surface_voxel_up_and_down/"

if not os.path.exists(save_path):
    os.makedirs(save_path)
#method 0: half voxel up and down (if spacing_mm is 0.12mm, then 0.06mm up and down)
#method 1: full voxel length along the surface normal (if spacing_mm is 0.12mm, then 0.12mm up and down)
#090925 DJ - Ting suggested to keep the spacing_mm as 0.24mm or 0.12mm and maximum steps at 8 or 16, respectively

PARAMS = {'spacing_mm': 0.12, 'dist_max_mm': 0.96, 'method':1}
#get all the folders (subjects) in the current directory
subject_folders = [f for f in os.listdir(path) if os.path.isdir(os.path.join(path, f))]


for subject_name in subject_folders:
    #check if there is a file that includes  "32k_fs_LR.surf.gii"
    #these files are calculated using the "calculate_surface_normal_and_coordinates.py" script
    #or you can just calculate them using workbench command*
    for layers in ["inf"]:
        regex_target_1 = f"{layers}.32k_fs_LR.surfnorm.func.gii"
        regex_target_2 = f"{layers}.32k_fs_LR.coord.func.gii"

        files = os.listdir(os.path.join(path, subject_name))
        matching_surfnorm = [f for f in files if regex_target_1 in f]
        matching_coord = [f for f in files if regex_target_2 in f]
        print(matching_surfnorm)
        print(matching_coord)
        
        if matching_surfnorm and matching_coord:
            for i,file in enumerate(matching_surfnorm):
                surfacenorm_file = os.path.join(path, subject_name, matching_surfnorm[i])
                coord_file = os.path.join(path, subject_name, matching_coord[i])

                #get the hemisphere from the file name
                all_points, dist_array = get_surface_xyz(
                    surfacenorm_file,
                    coord_file,
                    params=PARAMS,
                    spacing_mm=PARAMS['spacing_mm'],
                    dist_max_mm=PARAMS['dist_max_mm']
                )
               
                hemi = matching_surfnorm[0].split('.')[0].split('_')[-1]
                #subject_name = 'I48_new_confidence'
                tmp = nib.load(os.path.join(path, subject_name, f'{hemi}.inf.32k_fs_LR.surf.gii'))

                # Create output directory once
                output_dir = os.path.join(save_path, subject_name, f'surf_voxel_up_and_down', f'{int(PARAMS["spacing_mm"]*1000)}um_method{PARAMS["method"]}')
                os.makedirs(output_dir, exist_ok=True)

                for j in range(len(all_points)):
                    # Force assignment using setfield
                    new_data = np.array(all_points[j], dtype=tmp.darrays[0].data.dtype)
                    tmp.darrays[0].__dict__['data'] = new_data

                    output_file = os.path.join(output_dir, f'{hemi}.inf.32k_fs_LR_{dist_array[j]:.2f}.surf.gii')
                    nib.save(tmp, output_file)

pial.*\.32k_fs_LR\.surf\.gii
['lh.pial.32k_fs_LR.surf.gii']
Found /Users/dennis.jungchildmind.org/Desktop/To-be-uploaded-to-CMI-server/data/thickness_data//exvivo_final/I64/lh.pial.32k_fs_LR.surf.gii in I64
Surface normal output file: /Users/dennis.jungchildmind.org/Desktop/To-be-uploaded-to-CMI-server/data/thickness_data//exvivo_final/I64/lh.pial.32k_fs_LR.surfnorm.func.gii
Coordinate output file: /Users/dennis.jungchildmind.org/Desktop/To-be-uploaded-to-CMI-server/data/thickness_data//exvivo_final/I64/lh.pial.32k_fs_LR.coord.func.gii
Successfully calculated surface normals and coordinates for I64
white.*\.32k_fs_LR\.surf\.gii
['lh.white.32k_fs_LR.surf.gii']
Found /Users/dennis.jungchildmind.org/Desktop/To-be-uploaded-to-CMI-server/data/thickness_data//exvivo_final/I64/lh.white.32k_fs_LR.surf.gii in I64
Surface normal output file: /Users/dennis.jungchildmind.org/Desktop/To-be-uploaded-to-CMI-server/data/thickness_data//exvivo_final/I64/lh.white.32k_fs_LR.surfnorm.func.gii
Coordinate o